In [2]:
# properties
colors = 8
shapes = 12
locs = 9

configs = colors*shapes*locs
print(configs)

864


In [20]:
import math
for n in range(1,8):
    print("situations(n_objects={}): {:>27,}".format(n, int(math.pow(configs,n))))

situations(n_objects=1):                         864
situations(n_objects=2):                     746,496
situations(n_objects=3):                 644,972,544
situations(n_objects=4):             557,256,278,016
situations(n_objects=5):         481,469,424,205,824
situations(n_objects=6):     415,989,582,513,831,936
situations(n_objects=7): 359,414,999,291,950,792,704


In [19]:
# For a single situation we still have multiple refs ! How many refs do we have on average?

